In [1]:
import json
import pandas as pd
from pandas import json_normalize
from pprint import pprint
import sqlite3 
conn = sqlite3.connect('test.sqlite')

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [2]:

with open('parks_fs.json') as f:
    fs = json.load(f)
pprint (fs)

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 39.703014,
                                                  'longitude': -105.078792},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
                                       'suffix': '.png'},
                              'id': 16032,
                              'name': 'Park'}],
              'chains': [],
              'distance': 312,
              'fsq_id': '4ed022f89a52262b82749115',
              'geocodes': {'main': {'latitude': 39.704233,
                                    'longitude': -105.08234},
                           'roof': {'latitude': 39.704233,
                                    'longitude': -105.08234}},
              'link': '/v3/places/4ed022f89a52262b82749115',
              'location': {'census_block': '080590117014002',
                           'country': 'US',
          

Parse through the response to get the POI details you want (rating, name, location, etc)

In [3]:
from pandas import json_normalize
json_normalize(fs)
results = json_normalize(fs, record_path='results')
categories = json_normalize(fs['results'], record_path='categories', errors='ignore')

results.rename(columns = {'name':'park_name', 'location.formatted_address':'location_address'}, 
               inplace = True)
categories = json_normalize(fs['results'], record_path='categories', 
                            meta=['park_name','location_address'], errors='ignore')

fs_park_results = results[['park_name','location_address', 'distance']]
fs_park_results.head()

,park_name,location_address,distance
0,Bell Mar Park,"Lakewood, CO 80226",312
1,Belmar Park Running Trail,"Lakewood, CO 80226",622
2,Skate Street Skate Park,"Lakewood, CO 80232",964
3,Lakewood Recreation Department,"777 S Wadsworth Blvd, Lakewood, CO 80226",362
4,Belmar Dog Park,"501 S Teller Ct, Lakewood, CO 80226",364


In [4]:
categorylist = []
for i in range(len(results['park_name'])):
    categorylist.append(fs['results'][i]['categories'][0]['name'])

cat_list = pd.DataFrame(categorylist)
cat_list.columns = ['category']

Put your parsed results into a DataFrame

In [5]:
parks = pd.concat([fs_park_results, cat_list], axis=1, join='inner')
parks.sort_values(by = 'distance')

,park_name,location_address,distance,category
0,Bell Mar Park,"Lakewood, CO 80226",312,Park
3,Lakewood Recreation Department,"777 S Wadsworth Blvd, Lakewood, CO 80226",362,Park
4,Belmar Dog Park,"501 S Teller Ct, Lakewood, CO 80226",364,Dog Park
1,Belmar Park Running Trail,"Lakewood, CO 80226",622,Park
2,Skate Street Skate Park,"Lakewood, CO 80232",964,Park


In [6]:
parks['park_name'] = parks['park_name'].astype('str')
parks['location_address'] = parks['location_address'].astype('str')
parks.to_sql(name='parks_fs', con=conn, if_exists='replace', index=False)
pd.read_sql('select * from parks_fs', conn)

,park_name,location_address,distance,category
0,Bell Mar Park,"Lakewood, CO 80226",312,Park
1,Belmar Park Running Trail,"Lakewood, CO 80226",622,Park
2,Skate Street Skate Park,"Lakewood, CO 80232",964,Park
3,Lakewood Recreation Department,"777 S Wadsworth Blvd, Lakewood, CO 80226",362,Park
4,Belmar Dog Park,"501 S Teller Ct, Lakewood, CO 80226",364,Dog Park


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [7]:
with open('parks_yelp.json') as y:
    yelp = json.load(y)
pprint (yelp)

{'businesses': [{'alias': 'belmar-park-lakewood',
                 'categories': [{'alias': 'parks', 'title': 'Parks'}],
                 'coordinates': {'latitude': 39.7041997113815,
                                 'longitude': -105.087161511183},
                 'display_phone': '(303) 987-7800',
                 'distance': 728.0391226802858,
                 'id': 'Dny5Xe5XNvjR6xESa2Ej0A',
                 'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/mv5zlINq0eRuMx6DPhibwQ/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '801 S Yarrow St',
                              'address2': '',
                              'address3': '',
                              'city': 'Lakewood',
                              'country': 'US',
                              'display_address': ['801 S Yarrow St',
                                                  'Lakewood, CO 80226'],
                              'state': 'CO',
                         

Parse through your result and get POI details

In [8]:
from pandas import json_normalize
json_normalize(yelp)
results = json_normalize(yelp, record_path='businesses')
results.rename(columns = {'name':'park_name', 'location.display_address':'location_address'}, 
               inplace = True)
yelp_park = results[['park_name','location_address','distance','rating','review_count']]
yelp_park

,park_name,location_address,distance,rating,review_count
0,Belmar Park,"[801 S Yarrow St, Lakewood, CO 80226]",728.039123,4.0,9


In [9]:
categorylist = []
for i in range(len(results['park_name'])):
    categorylist.append(yelp['businesses'][i]['categories'][0]['title'])

cat_list = pd.DataFrame(categorylist)
cat_list.columns = ['category']

Put your parsed results into a DataFrame

In [10]:
parks = pd.concat([yelp_park, cat_list], axis=1, join='inner')
parks['distance'] = round(parks['distance'])
parks.sort_values(by = 'distance')


,park_name,location_address,distance,rating,review_count,category
0,Belmar Park,"[801 S Yarrow St, Lakewood, CO 80226]",728.0,4.0,9,Parks


In [11]:
parks['park_name'] = parks['park_name'].astype('str')
parks['location_address'] = parks['location_address'].astype('str')
parks.to_sql(name='parks_yelp', con=conn, if_exists='replace', index=False)
pd.read_sql('select * from parks_yelp', conn)

,park_name,location_address,distance,rating,review_count,category
0,Belmar Park,"['801 S Yarrow St', 'Lakewood, CO 80226']",728.0,4.0,9,Parks


# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Get the top 10 restaurants according to their rating

# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).